In [36]:
import os
os.chdir('../src/models')

FileNotFoundError: [WinError 3] The system cannot find the path specified: '../src/models'

In [43]:
import pandas as pd
from datetime import datetime

real_hosp = pd.read_csv('../../data/raw/hopitalized_2020-07-24.csv')
real_hosp = real_hosp[real_hosp['type'] == 'srag'].copy()
real_hosp.date = pd.to_datetime(real_hosp.date, format='%Y-%m-%d')
earlier = pd.DataFrame([[d, 0, 0, 0, 0, 'srag'] for d in [datetime(2020, 2, x) for x in range(15, 30)]], 
                       columns = real_hosp.columns)
real_hosp = pd.concat([earlier, real_hosp]).reset_index(drop=True)

real_covid = pd.read_csv('../../data/raw/hopitalized_2020-07-24.csv')
real_covid = real_covid[real_covid['type'] == 'covid'].copy()
real_covid.date = pd.to_datetime(real_covid.date, format='%Y-%m-%d')
earlier = pd.DataFrame([[d, 0, 0, 0, 0, 'srag'] for d in [datetime(2020, 2, x) for x in range(15, 30)]], 
                       columns = real_covid.columns)
real_covid = pd.concat([earlier, real_covid]).reset_index(drop=True)

mobility_trends = pd.read_csv('../../data/raw/Global_Mobility_Report.csv', low_memory=False)
mobility_trends = mobility_trends[(mobility_trends.country_region_code == 'BR') &
                (mobility_trends.sub_region_1 == 'State of São Paulo')]
mobility_trends = mobility_trends[(mobility_trends.sub_region_2 == 'São Paulo')].reset_index(drop=True).copy()
mobility_trends.date = pd.to_datetime(mobility_trends.date, yearfirst=True, dayfirst=False)
mobility_trends['neighbor'] = -1*mobility_trends[['retail_and_recreation_percent_change_from_baseline',
                                                'grocery_and_pharmacy_percent_change_from_baseline',
                                                'parks_percent_change_from_baseline']].mean(axis=1)/100
mobility_trends['home'] = -1*mobility_trends['residential_percent_change_from_baseline']/100
mobility_trends['work'] = -1*mobility_trends['workplaces_percent_change_from_baseline']/100
def school_trend(x):
    if x > datetime(2020, 3, 22):
        return 1
    elif x.weekday() < 5:
        return 0
    else:
        return .8

mobility_trends['school'] = mobility_trends['date'].apply(school_trend)

In [27]:
set(real_hosp.date) - set(mobility_trends.date)

set()

In [28]:
set(mobility_trends.date) - set(real_hosp.date)

{Timestamp('2020-08-21 00:00:00'),
 Timestamp('2020-08-22 00:00:00'),
 Timestamp('2020-08-23 00:00:00'),
 Timestamp('2020-08-24 00:00:00'),
 Timestamp('2020-08-25 00:00:00'),
 Timestamp('2020-08-26 00:00:00'),
 Timestamp('2020-08-27 00:00:00'),
 Timestamp('2020-08-28 00:00:00'),
 Timestamp('2020-08-29 00:00:00'),
 Timestamp('2020-08-30 00:00:00'),
 Timestamp('2020-08-31 00:00:00'),
 Timestamp('2020-09-01 00:00:00'),
 Timestamp('2020-09-02 00:00:00'),
 Timestamp('2020-09-03 00:00:00'),
 Timestamp('2020-09-04 00:00:00'),
 Timestamp('2020-09-05 00:00:00'),
 Timestamp('2020-09-06 00:00:00')}

In [24]:
mobility_trends

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,neighbor,home,work,school
0,BR,Brazil,State of São Paulo,São Paulo,NaN,NaN,NaN,2020-02-15,13.0,10.0,40.0,18.0,7.0,-2.0,-0.210000,0.02,-0.07,0.8
1,BR,Brazil,State of São Paulo,São Paulo,NaN,NaN,NaN,2020-02-16,7.0,8.0,43.0,13.0,0.0,0.0,-0.193333,-0.00,-0.00,0.8
2,BR,Brazil,State of São Paulo,São Paulo,NaN,NaN,NaN,2020-02-17,-1.0,1.0,0.0,11.0,15.0,-2.0,-0.000000,0.02,-0.15,0.0
3,BR,Brazil,State of São Paulo,São Paulo,NaN,NaN,NaN,2020-02-18,-1.0,-2.0,-1.0,11.0,14.0,-2.0,0.013333,0.02,-0.14,0.0
4,BR,Brazil,State of São Paulo,São Paulo,NaN,NaN,NaN,2020-02-19,-2.0,0.0,3.0,11.0,13.0,-2.0,-0.003333,0.02,-0.13,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,BR,Brazil,State of São Paulo,São Paulo,NaN,NaN,NaN,2020-09-02,-22.0,16.0,6.0,-31.0,-27.0,13.0,-0.000000,-0.13,0.27,1.0
201,BR,Brazil,State of São Paulo,São Paulo,NaN,NaN,NaN,2020-09-03,-19.0,22.0,15.0,-28.0,-27.0,13.0,-0.060000,-0.13,0.27,1.0
202,BR,Brazil,State of São Paulo,São Paulo,NaN,NaN,NaN,2020-09-04,-20.0,23.0,20.0,-24.0,-24.0,12.0,-0.076667,-0.12,0.24,1.0
203,BR,Brazil,State of São Paulo,São Paulo,NaN,NaN,NaN,2020-09-05,-11.0,31.0,4.0,-16.0,-5.0,5.0,-0.080000,-0.05,0.05,1.0


In [6]:
import plotly.graph_objects as go

fig = go.Figure()

x = pd.date_range(datetime(2020, 2, 15), periods=len(mobility_trends))


fig.add_trace(go.Scatter(x = x, y = 1-1*mobility_trends['school'],
                         name='Escolas', opacity=0.7))
fig.add_trace(go.Scatter(x =x, y = 1-1*mobility_trends['neighbor'],
                         name='Vizinhanças', opacity=0.7))
fig.add_trace(go.Scatter(x = x, y = 1-1*mobility_trends['work'],
                         name='Trabalhos', opacity=0.7))
fig.add_trace(go.Scatter(x = x, y = 1-1*mobility_trends['home'],
                         name='Residências', opacity=0.7))
fig.update_layout(
            #title='Google Mobility Trends',
            xaxis =dict(
            tickformat= '%b',
            tickvals = [datetime(2020, x, 15) for x in range(2,9)],
            ))
fig.add_trace(go.Scatter(x=2*[pd.datetime(2020, 3, 21)],
                             y=([0]+[1.4]),
                             mode='lines',
                             #yaxis="y2", 
                             line_width=3, line_dash='dot',
                             line_color='gray', opacity=1,                             
                             name='Início da Quarentena',
                             showlegend=True))
fig.update_yaxes(title='Nível de mobilidade relativa')
fig.update_layout(
    font=dict(color="black",
            size=24),
    margin=dict(
            l=50,
            r=20,
            b=50,
            t=50,
            pad=0),
    legend=dict(
            x=.75,
            y=0.1,
            traceorder="normal",
            font=dict(
                size=16,
                color="black", family='sans-serif'
            ),
            bgcolor="white",
            bordercolor="Black",
            borderwidth=2
        ),
    )
fig.update_layout( 
        xaxis={'showgrid': False,},
        yaxis = {'showgrid': False, 'zeroline': False},
        hovermode="x",
        font=dict(family='sans-serif'),
        plot_bgcolor='rgb(255, 255, 255)',
#        title={
#        'text': 'Google Mobility Trends',
#        'y':.98,
#        'x':0.5,
#        'xanchor': 'center',
#        'yanchor': 'top',
#        'font':{'size': 36}},
)
fig.write_image(f"mobility_trends.pdf")
#fig.show()

c:\anaconda\envs\cmdp\lib\site-packages\ipykernel_launcher.py:22: FutureWarning:

The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.



In [7]:
import simulate_pandemic as simp
from actions import costs, city_restrictions
from MCFS import mcts, treeNode
from CMDP import CovidState

from tqdm import tqdm
import numpy as np

Loading Graph...Done!


In [8]:
mobility_trends.to_csv('mobility_restrictions.csv', sep=';', index=False)

In [9]:
def real_policy(day):
    restrictions = {'work': mobility_trends['work'].iloc[day], 
                    'school': mobility_trends['school'].iloc[day],   
                    'home': 0, 
                    'neighbor':  mobility_trends['neighbor'].iloc[day]}
    
    return restrictions

def unrestricted(day):
    return {'work': 0,  'school': 0,   'home': 0, 'neighbor': 0}

In [10]:
def run_simulation(policy, step_size = 7, days = 364):
    pop_matrix = simp.init_infection(5e-5)
    print(np.sum(pop_matrix[:,1] == 1))

    data = []
    restrictions = policy(0)

    for day in tqdm(range(1, min(len(mobility_trends), len(real_hosp))-2)):
        #if less than 20% still susceptible, break simulation
        if np.sum(pop_matrix[:,1] == -1) > pop_matrix.shape[0]*.9: break
            
        restrictions = policy(day)

        pop_matrix = simp.spread_infection(pop_matrix, restrictions, day)
        pop_matrix = simp.lambda_leak_expose(pop_matrix, day)
        pop_matrix = simp.update_population(pop_matrix)
            
        data.append(pd.Series(pop_matrix[:,1]).value_counts())
        #data.append(np.array(sorted(pop_matrix, key=lambda x: x[0]))[:, 1])

    #return data, actions, contacts_infected, pop_matrix
    return data

In [30]:
from joblib import Parallel, delayed
sims = Parallel(n_jobs=-1)(delayed(run_simulation)(real_policy, step_size=1, days=len(real_hosp)+1) for i in range(100))

#data = [run_simulation(real_policy, step_size=1, days=100) for i in range(10)]

In [14]:
results = [pd.DataFrame([d for d in s]).fillna(0) for s in tqdm(sims)]
df_mean = pd.concat([r for r in results]).groupby(level=0).mean()
df_std = pd.concat([r for r in results]).groupby(level=0).std()
#data = pd.DataFrame(data).fillna(0)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:03<00:00, 25.06it/s]


In [44]:
import plotly.graph_objects as go

def make_beds_graph(data, std, actions, step_size, title, color_map=None, make_df=True):
    fig = go.Figure()
    
    if make_df:
        data = pd.DataFrame([pd.Series(d).value_counts() for d in data])
        data.fillna(0, inplace=True)

    color_map = {
        'Lockdown':          'rgb(0.83, 0.13, 0.15)',
        'Hard Quarantine':    'rgb(0.85, 0.35, 0.13)',
        'Light Quarantine':   'rgb(0.97, 0.91, 0.56)',
        'Social Distancing':  'rgb(0.67, 0.88, 0.69)',
        'Unrestricted':        'rgb(0.86, 0.86, 0.86)'    
    }

    pop = 55492

    actions = list(map(color_map.get,  actions))

    x = pd.date_range(datetime(2020, 2, 15), periods=len(data[0]))


    # HOSPITALIZED LINE #####################################################################################
    #\definecolor{royalblue(web)}{rgb}{0.25, 0.41, 0.88}
    fig.add_trace(go.Scatter(x=x, y=12e6*data[3]/pop, name='Simulado', line_color = 'rgb(0.25, 0.41, 0.88)',
                            line=dict(width=4), 
                            # yaxis="y2"
                            ))
    
    fig.add_trace(go.Scatter(x=x, y=(12e6*data[3]/pop) + (12e6*std[3]/pop), fill=None, mode='lines', showlegend=False,
                            #yaxis="y2",
                            line_width=0,
                            opacity = 0.5, line_color='rgb(0.45, 0.61, 0.99)'))
    
    
    fig.add_trace(go.Scatter(x=x, y=(12e6*data[3]/pop), fill='tonexty', mode='lines', 
                             #yaxis="y2",
                             opacity=0.5, line_color='rgb(0.45, 0.61, 0.99)', showlegend=False
                            # name="Standard Deviation"
                            ))
    
    fig.add_trace(go.Scatter(x=x, y=(12e6*data[3]/pop) - (12e6*std[3]/pop), fill=None, mode='lines', 
                             #yaxis="y2",
                             line_width=0,
                             line_color='rgb(0.45, 0.61, 0.99)',    showlegend=False))


    fig.add_trace(go.Scatter(x=x, y=(12e6*data[3]/pop), fill='tonexty', mode='lines',
                             #yaxis="y2",
                             line_color='rgb(0.45, 0.61, 0.99)', opacity=0.5, showlegend=False))
    
    # REAL DATA #################################################################################
    fig.add_trace(go.Scatter(x=x, y=real_hosp['observed'], name='Observado',
                        line=dict(width=3.5), line_color='black'
                        #yaxis="y2"
                        ))
    fig.add_trace(go.Scatter(x=x, y=real_covid['observed'], name='Covid',
                        line=dict(width=3.5), line_color='Red'
                        #yaxis="y2"
                        ))
    
    #fig.add_trace(go.Scatter(x=x, y=real_hosp['upper'], name='real_data_upper',
    #                        line=dict(width=3.5),
                            #yaxis="y2"
    #                        ))
    #fig.add_trace(go.Scatter(x=x, y=real_hosp['lower'], name='real_data_lower',
    #                        line=dict(width=3.5),
    #                        #yaxis="y2"
    #                        ))
    
    # CAPACITY LINE ################################################################################
    fig.add_trace(go.Scatter(x=x, y=len(data)*[12e6*0.0015], name='Capacidade', line_color = 'black',
                            line=dict(dash='dash', width = 2)
                             #, yaxis="y2"
                            ))
    #################################################################################################
    
    # ACTIONS BACKGROUND################################################################################
    """    fig.update_layout(
        shapes=[
            dict(
                type="rect",
                # x-reference is assigned to the x-values
                xref="x",
                # y-reference is assigned to the plot paper [0,1]
                yref="paper",
                x0=(step_size*i),
                y0=0,
                x1=step_size*(i+1),
                y1=1,
                fillcolor=a,
                opacity=0.45,
                layer="below",
                line_width=0,
            ) for i,a in enumerate(actions)] 
    )"""

    # GHOST TRACES FOR LEGEND#############################
    #for k,v in color_map.items():
    #    fig.add_trace(go.Bar(x=[None], y=[None], marker=dict(color=v), name = k))

    fig.update_layout( xaxis={'showgrid': False,},
                      yaxis = {'showgrid': False, 'zeroline': False,  
                               'title':'Número de Hospitalizados',
                              'tickfont':{'size': 26},'dtick': 6000,},  hovermode="x",
                      font=dict(family='sans-serif'),
                    )
    
    
    fig.update_layout(
        #xaxis = dict(
        #    tickmode = 'array',
        #    tickvals = list(range(0, len(data), 14)),
        #    ticktext = list(range(0, int(len(data)/7), 2)),
        #    title = 'Weeks'
        #    ),
        plot_bgcolor='rgb(225, 229, 234)',
        showlegend=True,
        font=dict(
            color="black",
            size=24
        ),
        xaxis =dict(tickfont=dict(
                size=26,
                color="black",
            ), 
            tickformat= '%b',
            tickvals = [datetime(2020, x, 15) for x in range(2,9)],
            ),
        margin=dict(
            l=50,
            r=20,
            b=50,
            t=50,
            pad=0
    ))
    
    fig.update_layout(
       # title={
       #     'text': 'Validação do Modelo',
       #     'y':.98,
       #     'x':0.5,
       #     'xanchor': 'center',
       #     'yanchor': 'top',             'font':{'size': 36}}
    )

    
    fig.update_layout(
        legend=dict(
            x=.02,
            y=0.85,
            traceorder="normal",
            font=dict(
                size=28,
                color="black", family='sans-serif'
            ),
            bgcolor="white",
            bordercolor="Black",
            borderwidth=2
        )
    )

    #fig.update_layout(yaxis_type="log")
    #fig.update_yaxes(automargin=True)
    fig.write_image(f"{title}.pdf")

    fig.show()

In [45]:
make_beds_graph(df_mean, df_std, len(df_mean)*['Unrestricted'], 1, 'SRAG_Real_Data', make_df=False)

In [19]:
1 - df_mean.iloc[83][0] / df_mean.iloc[0][0]

0.06482473372141218